This examples shows how to use the python API to:

1. Create a fleet of electric vehicles
1. Create a charging-post infrastructure
1. Create a constraint
1. Allocate fleet to infrastructure under some constraints
1. Plot the density profile of distances from vehicle destination to charging post

# Setting up the fleet, infrastructure, and constraint
First we create the fleet. It is simply a pandas data-frame with a specific set of
attributes (one attribute per column).  More attributes/columns can be added if
required for a specific application.  Here, we generate the fleet randomly with a
fairly minimal set. Run `evosim.fleet.random_fleet?` in a code-cell to see how to
parametrize the random generation. The ``seed`` argument is optional. We give it here
to ensure this notebook runs reproducibly during the tests and does not interfere with
version control.

In [1]:
import numpy as np

import evosim

rng = np.random.default_rng(2)
fleet = evosim.fleet.random_fleet(1000, seed=rng)
fleet

,latitude,longitude,socket,charger,dest_lat,dest_long,model
0,51.37,0.93,CHADEMO,SLOW,51.55,0.96,SMART_EQ_FORTWO
1,51.38,0.26,CCS,FAST,51.49,0.04,VOLVO_V90_TWIN_ENGINE
2,51.62,0.13,TYPE1,FAST,51.39,1.14,TESLA_MODEL_S
3,51.29,-0.15,TYPE1,SLOW,51.32,-0.28,BMW_X5_40E
4,51.52,0.90,THREE_PIN_SQUARE,FAST,51.28,1.00,NISSAN_LEAF
5,51.58,0.38,CHADEMO,FAST,51.55,0.36,BMW_225XE
6,51.33,1.03,CCS,SLOW,51.53,0.60,UNKNOWN
7,51.27,0.36,TYPE1,FAST,51.50,0.44,MERCEDES_BENZ_C350E
8,51.37,1.03,TYPE1,SLOW,51.51,0.13,HYUNDAI_IONIQ_ELECTRIC
9,51.55,0.76,TYPE1,RAPID,51.66,0.98,RENAULT_ZOE


Then we create the infrastructure of charging posts. Again, it is just a pandas
data-frame with a specific set of columns. And again, it is filled with mostly random
data. Arbitrarily, we've decided the capacity of the posts would range from 1 to 5 and
the occupancy from 0 to 2.

In [2]:
infrastructure = evosim.charging_posts.random_charging_posts(
    len(fleet) // 3, capacity=5, occupancy=2, seed=rng
)
infrastructure

,latitude,longitude,socket,charger,capacity,occupancy
0,51.39,-0.13,TYPE2,SLOW,3,2
1,51.36,-0.30,TYPE1,SLOW,1,0
2,51.26,1.20,TYPE1,SLOW,4,0
3,51.63,-0.48,DC_COMBO_TYPE2,SLOW,1,0
4,51.67,0.65,DC_COMBO_TYPE2,RAPID,4,0
5,51.41,0.05,DC_COMBO_TYPE2,FAST,1,0
6,51.31,0.50,DC_COMBO_TYPE2,FAST,4,1
7,51.53,0.92,DC_COMBO_TYPE2,SLOW,2,0
8,51.68,0.10,TYPE1,FAST,3,1
9,51.41,0.97,DC_COMBO_TYPE2,FAST,4,1


The random generation algorithm is smart enough to ensure the occupancy is always
smaller or equal to the capacity.

In [3]:
assert (infrastructure.occupancy <= infrastructure.capacity).all()

Now we can create a ``matcher`` function: a constraint which allows or disallows
matching an electric vehicle with a charging post. Individual matchers can be found in
``evosim.matchers``. But there is also a factory function which helps to generate a
matcher function that is a combination of individual matchers. They are combined with
``and`` logic: all individual constraints must be true for the match to be okayed.

In [4]:
matcher = evosim.matchers.factory(["socket_compatibility", "charger_compatibility"])

We can match a single vehicle to a single post

In [5]:
matcher(fleet.loc[0], infrastructure.loc[1])

<Sockets.0: 0>

Or the full fleet to a single post

In [6]:
matcher(fleet, infrastructure.loc[1])

0      False
1      False
2      False
3       True
4      False
5      False
6      False
7      False
8       True
9      False
10     False
11     False
12     False
13      True
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
       ...  
974    False
975    False
976    False
977    False
978    False
979    False
980    False
981    False
982    False
983    False
984    False
985    False
986    False
987    False
988    False
989    False
990    False
991    False
992    False
993    False
994    False
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

Or a single vehicle to the full infrastructure

In [7]:
matcher(fleet.loc[1], infrastructure)

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22      True
23     False
24     False
25     False
       ...  
307    False
308    False
309    False
310    False
311    False
312    False
313    False
314    False
315    False
316    False
317    False
318    False
319    False
320    False
321    False
322    False
323    False
324    False
325    False
326    False
327    False
328    False
329    False
330    False
331    False
332    False
Length: 333, dtype: bool

Or we can match fleet and infrastructure element-wise: the first vehicle to the first
post, the second to the second, and so on. However, this element-wise comparison only
works if there are as many rows in fleet as in infrastructure. At time of writing, the
error thrown by pandas is not particularly enlightening:

In [8]:
try:
    matcher(fleet, infrastructure)
except TypeError as e:
    print("Yup, we got an error: ", e)

Yup, we got an error:  unsupported operand type(s) for &: 'Sockets' and 'bool'


But we can make the comparison work as follows:

In [9]:
matcher(fleet.iloc[: len(infrastructure)], infrastructure)

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
       ...  
307    False
308     True
309    False
310    False
311    False
312     True
313    False
314    False
315    False
316    False
317    False
318    False
319    False
320    False
321    False
322    False
323    False
324    False
325    False
326    False
327    False
328    False
329    False
330    False
331    False
332    False
Length: 333, dtype: bool

# Running the allocation algorithm
The matcher can be used to constrain the allocation of vehicles to posts. Below, we
use the *random* allocator for this purpose. It allocates any vehicle to any post, as
long as the matcher okays it:

In [10]:
fleet_allocation = evosim.allocators.random_allocator(fleet, infrastructure, matcher)
fleet_allocation

,latitude,longitude,socket,charger,dest_lat,dest_long,model,allocation
0,51.37,0.93,CHADEMO,SLOW,51.55,0.96,SMART_EQ_FORTWO,110
1,51.38,0.26,CCS,FAST,51.49,0.04,VOLVO_V90_TWIN_ENGINE,163
2,51.62,0.13,TYPE1,FAST,51.39,1.14,TESLA_MODEL_S,81
3,51.29,-0.15,TYPE1,SLOW,51.32,-0.28,BMW_X5_40E,14
4,51.52,0.90,THREE_PIN_SQUARE,FAST,51.28,1.00,NISSAN_LEAF,151
5,51.58,0.38,CHADEMO,FAST,51.55,0.36,BMW_225XE,227
6,51.33,1.03,CCS,SLOW,51.53,0.60,UNKNOWN,<NA>
7,51.27,0.36,TYPE1,FAST,51.50,0.44,MERCEDES_BENZ_C350E,178
8,51.37,1.03,TYPE1,SLOW,51.51,0.13,HYUNDAI_IONIQ_ELECTRIC,307
9,51.55,0.76,TYPE1,RAPID,51.66,0.98,RENAULT_ZOE,214


# Analysis and plots
We are now in a position to analyse the results. First we will take a look at the
unallocated vehicles. Then we will plot the distribution of distances from each
vehicles destination to each charging post.
Allocated and unallocated vehicles can be obtained as:

In [11]:
unallocated_fleet = fleet[fleet_allocation.allocation.isna()]
allocated_fleet = fleet_allocation[~fleet_allocation.allocation.isna()]

We can update the infrastructure to account for newly occupied posts. Note that the
infrastructure is copied with ``deep=True`` to ensure that changing the occupancy in
``new_infrastructure`` does not affect the original dataframe.

In [12]:
new_infrastructure = infrastructure.copy(deep=True)
new_occupancy = allocated_fleet.groupby("allocation").allocation.count()
new_infrastructure.loc[new_occupancy.index, "occupancy"] += new_occupancy

Now we plot the number allocated and unallocated infrastructure and vehicles

In [13]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()
allocation_data = {
    "type": ["Initial Infrastructure", "Final Infrastructure", "Fleet"],
    "allocated": [
        infrastructure.occupancy.sum(),
        new_infrastructure.occupancy.sum(),
        len(allocated_fleet),
    ],
    "unallocated": [
        (infrastructure.capacity - infrastructure.occupancy).sum(),
        (new_infrastructure.capacity - new_infrastructure.occupancy).sum(),
        len(allocated_fleet),
    ],
}

barplot = figure(
    x_range=allocation_data["type"],
    height=200,
    title="Infrastructure and vehicle allocation",
)
barplot.vbar_stack(
    ["allocated", "unallocated"],
    x="type",
    width=0.9,
    source=allocation_data,
    color=["darkgrey", "lightgrey"],
    alpha=[1, 0.5],
    legend_label=["allocated", "unallocated"],
)
barplot.y_range.start = 0
barplot.legend.location = "top_left"
barplot.legend.orientation = "horizontal"
show(barplot)

Loading BokehJS ...

To plot the density profile, we need the distance from the vehicle destinations to the
allocated charging posts. We use the ``evosim``'s objective function for that purpose.
The objective function expects two dataframe arguments, each with a "latitude" and a
"longitude" column. Hence, we first rename the destination columns of the
``allocated_fleet``. Also note that since the objective function compares dataframes,
they must conform to the same index.

In [14]:
distance = evosim.objectives.distance(
    (
        allocated_fleet[["dest_lat", "dest_long"]].rename(
            columns=dict(dest_lat="latitude", dest_long="longitude")
        )
    ),
    infrastructure.loc[allocated_fleet.allocation].set_index(allocated_fleet.index),
)

density_plot = figure(title="Distance to charging post",)
y, x = np.histogram(distance, bins=20)
density_plot.line((x[1:] + x[:-1]) / 2, y)
show(density_plot)